## Pre-Processing

In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import warnings
warnings.filterwarnings("ignore")

In [98]:
## Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [99]:
## Drop unnecessary columns

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [100]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [101]:
## One-hot encode categorical variables
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [102]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [103]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [104]:
## Combine the encoded columns with the original dataset
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1) 
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [105]:
## Save the encoders
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [106]:
## Divide the dataset into features and target variable
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [107]:
## save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [108]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [109]:
## Build the ANN model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## Input layer with 64 neurons
    Dense(32, activation='relu'), ## Hidden layer with 32 neurons
    Dense(1, activation='sigmoid') ## Output layer for binary classification
])

In [110]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001) ## Define the optimizer

In [112]:
## Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
## Set up the tensorboard 
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [114]:
## Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [115]:
## Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    callbacks=[early_stopping, tensorflow_callback])

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4606 - accuracy: 0.7986 - val_loss: 0.3972 - val_accuracy: 0.8350
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3887 - accuracy: 0.8395 - val_loss: 0.3614 - val_accuracy: 0.8480
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3598 - accuracy: 0.8529 - val_loss: 0.3478 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3471 - accuracy: 0.8565 - val_loss: 0.3485 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3406 - accuracy: 0.8596 - val_loss: 0.3426 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3364 - accuracy: 0.8595 - val_loss: 0.3453 - val_accuracy: 0.8570
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3342 - accuracy: 0.8618 - val_loss: 0.3462 - val_accuracy: 0.8555

In [116]:
model.save('model.h5')  ## Save the trained model

In [117]:
## Load TensorBoard logs
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [118]:
%tensorboard --logdir logs/fit

Launching TensorBoard...